In [1]:
from keras import models, layers, activations, optimizers, losses
from keras.applications import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

/home/hamza/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
conv_base = VGG16(input_shape=(150,150,3), weights='imagenet', include_top=False)

In [3]:
base_dir = '/home/hamza/DL/dataset_cats_vs_dogs'
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')
test_dir = os.path.join(base_dir,'test')

train_cats_dir = os.path.join(train_dir,'cats')
validation_cats_dir = os.path.join(validation_dir,'cats')
test_cats_dir = os.path.join(test_dir,'cats')

train_dogs_dir = os.path.join(train_dir,'dogs')
validation_dogs_dir = os.path.join(validation_dir,'dogs')
test_dogs_dir = os.path.join(test_dir,'dogs')

In [4]:
def extract_features(directory, sample_count):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(directory, class_mode='binary', batch_size=20, target_size=(150,150))
    
    features = np.zeros((sample_count, 4,4,512))
    
    labels = np.zeros((sample_count))
    i = 0
    batch_size = 20
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        
        if i * batch_size >= sample_count:
            break
            
    return features, labels
    
    
    
train_features, labels   = extract_features(train_dir, 2000)
validation_features, val_labels  = extract_features(validation_dir, 1000)
x_test, y_test = extract_features(test_dir, 1000)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
x_test = np.reshape(x_test, (1000, 4 * 4 * 512))

In [11]:
def build_model():
    
    model = models.Sequential()
    model.add(layers.Dense(256, activation=activations.relu, input_dim=4*4*512))
    model.add(layers.Dropout(0.5))    
    model.add(layers.Dense(1, activation=activations.sigmoid))
    return model

In [12]:
model = build_model()

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss=losses.binary_crossentropy, metrics=['acc'])

In [15]:
history = model.fit(train_features, 
                    labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, val_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 1s 534us/step - loss: 0.6031 - acc: 0.6745 - val_loss: 0.4579 - val_acc: 0.8250
Epoch 2/30
2000/2000 [==============================] - 1s 324us/step - loss: 0.4443 - acc: 0.7915 - val_loss: 0.3688 - val_acc: 0.8600
Epoch 3/30
2000/2000 [==============================] - 1s 378us/step - loss: 0.3626 - acc: 0.8440 - val_loss: 0.3245 - val_acc: 0.8750
Epoch 4/30
2000/2000 [==============================] - 1s 351us/step - loss: 0.3138 - acc: 0.8745 - val_loss: 0.2980 - val_acc: 0.8840
Epoch 5/30
2000/2000 [==============================] - 1s 346us/step - loss: 0.2927 - acc: 0.8845 - val_loss: 0.2853 - val_acc: 0.8870
Epoch 6/30
2000/2000 [==============================] - 1s 340us/step - loss: 0.2627 - acc: 0.8965 - val_loss: 0.2709 - val_acc: 0.8950
Epoch 7/30
2000/2000 [==============================] - 1s 318us/step - loss: 0.2518 - acc: 0.9010 - val_loss: 0.2635 - val_acc: 0.9000


In [16]:
history.history.keys()

dict_keys(['val_acc', 'acc', 'val_loss', 'loss'])

In [ ]:
h = history.history
h['val_acc'][-1]

In [19]:
import matplotlib.pyplot as plt
%matplotlib

acc  =   history.history['acc']
val_acc  =  history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Using matplotlib backend: TkAgg
